1RSB
===

\begin{align}
p(f) = & \begin{cases}
\frac{1}{2}\left\{ \prod_{i=1}^k\left(2\sum_{f'\geq f}q_i\left(f'\right)\right)-\prod_{i=1}^k\left(2\sum_{f'>f}q_i\left(f'\right)\right)\right\}  & \text{ for }f>0\\
1 - \prod_{i=1}^k\left(1-q_i(0)\right) & \text{ for }f=0
\end{cases}
\end{align}

In [83]:
∏ = prod
∑ = sum

function iterffixed(Q, N)
    p = OffsetVector(zeros(2N+1), -N:N)
    for f = 1:N-1
        v = ∏(∑(q[f:N]) for q ∈ Q) - ∏(∑(q[f+1:N]) for q ∈ Q)
        p[+f] += v/2
        p[-f] += v/2
    end
    p[0] = 1-prod(1-q[0] for q ∈ Q)
    p ./ sum(p)
end

function ⊛(p1, p2)
    N2 = max(lastindex(p1), lastindex(p2))
    q = OffsetArray(zeros(2N2+1),-N2:N2)
    for f1 in eachindex(p1)
        for f2 in eachindex(p2)
            q[clamp(f1+f2, -N2, N2)] += p1[f1]*p2[f2]
        end
    end
    q
end

function itergfixed(P, N)
    q = OffsetVector(zeros(2N+1), -N:N)
    q[-1:1] += [0.5, 0, 0.5]
    for p ∈ P
        q = q ⊛ p
    end
    q ./ sum(q)
end

itergfixed (generic function with 2 methods)

In [84]:
uni(N) = (p=OffsetVector(ones(2N+1), -N:N); p ./= sum(p))
residual(x) = (p=OffsetVector((x .* eachindex(x))[1:end], 0:lastindex(x)-1); p./=sum(p))

function RSB(Λ, K; maxiter=100, N=10)
    Λ1 = residual(Λ)
    K1 = residual(K)
    popP = [uni(N) for i=1:1000]
    popQ = [uni(N) for i=1:1000]

    distΛ1 = DiscreteNonParametric(collect(eachindex(Λ1)), Λ1.parent)
    distK1 = DiscreteNonParametric(collect(eachindex(K1)), K1.parent)

    for t = 1:maxiter
        for i = 1:1000
            d = rand(distΛ1)
            k = rand(distK1)
            Q = rand(popQ, d)
            P = rand(popP, k)
            popQ[i] = itergfixed(P, N)
            popP[i] = iterffixed(Q, N)
        end
    end
    popP, popQ
end

RSB (generic function with 1 method)

In [87]:
Λ = OffsetVector([0,0,0.5,0.5], 0:3)
K = OffsetVector([0,0,0,1], 0:3)


popP, popQ = RSB(Λ,K; maxiter=10);

In [88]:
popP

1000-element Vector{OffsetVector{Float64,Vector{Float64}}}:
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5  …  0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5  …  0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5  …  0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2421420758637933  …  0.2421420758637933, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.125, 0.0, 0.375  …  0.375, 0.0, 0.125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5  …  0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5  …  0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.014137308905222969, 0.017289075919916217, 0.20894909306926782  …  0.20894909306926782, 0.017289075919916217, 0.01413730890522296